# 測試 CkipNLP 套件的使用
### 在這個筆記本中成功實作了載入json中的新聞內容，
### 進行NER實體命名識別
### 篩選重複出現2次以上的指定類別的關鍵詞彙

In [ ]:
## 安裝指令
# pip install pandas
# pip install ckipnlp
# pip install "ckipnlp[tagger]" 或 pip install "ckipnlp[tagger-gpu]"

In [1]:
import pandas as pd
import logging
from ckipnlp.pipeline import CkipPipeline, CkipDocument
from ckipnlp.container.text import TextParagraph
import re

# Read database

In [4]:
database = pd.read_json("../../assets/json/test186.json")
database = database.transpose()
database_series = pd.Series(database["content"])

# Verify database

In [5]:
print(type(database))
print(database.head(1))
database["content"].head()

print(database_series[0])

<class 'pandas.core.frame.DataFrame'>
     source                  title  \
0  wantrich  第7波疫情來臨前 日本實施觀光區域優惠方案   

                                                link        date category  \
0  https://wantrich.chinatimes.com/news/202204049...  1649078642     None   

                                             content summary  
0  日本部分地區為拼經濟，原本去年4月起實施的「縣民優惠」措施，4月1日起擴大為「區域優惠」方案...    None  
日本部分地區為拼經濟，原本去年4月起實施的「縣民優惠」措施，4月1日起擴大為「區域優惠」方案；部分縣府主管說，盼趁著第7波疫情來臨前趕緊實施，降低對觀光業的衝擊。
日本讀賣新聞報導，COVID-19（2019冠狀病毒疾病）疫情對觀光業帶來嚴重打擊，日本有些縣的優惠措施，僅限居住在縣內或鄰縣的民眾，但1日起擴大為區域優惠，平均每人每晚住宿費最多可獲7000日圓（約新台幣1665元）補助，一直到28日為止。
區域優惠的範圍分6大區，北海道暨東北（涵蓋北海道、青森、岩手、宮城、秋田、山形、福島縣）、關東（涵蓋茨城、櫪木、群馬、埼玉、千葉、東京、神奈川、山梨）、北陸信越暨中部（涵蓋新潟、富山、石川、長野、福井、岐阜、靜岡、愛知、三重），其他還有近畿、中國暨四國、九州暨沖繩等區域。
日本為了振興經濟，2020年7月22日起實施「來去旅行」（Go To Travel）措施，但隨著疫情擴大，這項措施於2020年底停辦。縣民優惠或區域優惠措施被視為「來去旅行」措施的替代方案。不過，有些地方政府認為疫情未歇，對於實施這優惠措施仍持慎重態度。
縣民優惠措施於去年4月實施，觀光客每人住宿一晚的住宿費半價（上限5000日圓），同時還可獲得餐飲店等可使用的折價券（上限2000日圓）。各都道府縣獲得中央政府的補助金，推動縣民優惠措施。
日本觀光廳（相當於觀光局）3月25日宣布縣民優惠措施的對象擴大為全國6大區域。觀光

# Initialize pipeline from CKIPNLP

In [6]:
# options for enabling cuda, optional
pipeline = CkipPipeline(opts = {'word_segmenter': {'disable_cuda': False}, 'Ner Chunker': {'disable_cuda': False}})
# pipeline = CkipPipeline()

paragraph = TextParagraph().from_list(database_series.tolist())
#doc = CkipDocument(text=paragraph)
doc = CkipDocument(text=paragraph)

In [7]:
doc.text[:2]

TextParagraph(['日本部分地區為拼經濟，原本去年4月起實施的「縣民優惠」措施，4月1日起擴大為「區域優惠」方案；部分縣府主管說，盼趁著第7波疫情來臨前趕緊實施，降低對觀光業的衝擊。\n日本讀賣新聞報導，COVID-19（2019冠狀病毒疾病）疫情對觀光業帶來嚴重打擊，日本有些縣的優惠措施，僅限居住在縣內或鄰縣的民眾，但1日起擴大為區域優惠，平均每人每晚住宿費最多可獲7000日圓（約新台幣1665元）補助，一直到28日為止。\n區域優惠的範圍分6大區，北海道暨東北（涵蓋北海道、青森、岩手、宮城、秋田、山形、福島縣）、關東（涵蓋茨城、櫪木、群馬、埼玉、千葉、東京、神奈川、山梨）、北陸信越暨中部（涵蓋新潟、富山、石川、長野、福井、岐阜、靜岡、愛知、三重），其他還有近畿、中國暨四國、九州暨沖繩等區域。\n日本為了振興經濟，2020年7月22日起實施「來去旅行」（Go To Travel）措施，但隨著疫情擴大，這項措施於2020年底停辦。縣民優惠或區域優惠措施被視為「來去旅行」措施的替代方案。不過，有些地方政府認為疫情未歇，對於實施這優惠措施仍持慎重態度。\n縣民優惠措施於去年4月實施，觀光客每人住宿一晚的住宿費半價（上限5000日圓），同時還可獲得餐飲店等可使用的折價券（上限2000日圓）。各都道府縣獲得中央政府的補助金，推動縣民優惠措施。\n日本觀光廳（相當於觀光局）3月25日宣布縣民優惠措施的對象擴大為全國6大區域。觀光客在獲得出發地（居住地）到旅遊目的地（住宿地）的都道府縣彼此的同意之下，在區域內觀光者可獲得補助。觀光客利用此優惠措施，須持有打3劑疫苗或篩檢呈現陰性反應的證明。\n山梨縣對於縣民優惠或區域優惠措施寄予厚望。山梨縣的觀光客當中，約7成是來自東京都、神奈川縣等關東地區的人士。觀光廳在宣布優惠措施擴大為區域後，山梨縣在3月底前就與關東區域的茨城、群馬、埼玉等各縣達成適用優惠措施的協議。\n山梨縣政府的主管表示，希望趁第7波疫情來臨前，儘速實施區域優惠措施。\n埼玉縣除了最多給予觀光者7000日圓之外，不論是來自日本哪裡的觀光客，只要下榻縣指定的住宿設施，平均每晚可獲得最多價值3000日圓的優惠券。\n埼玉縣秩父市的旅館「湯之宿 和銅」主管表示，4月1日早上開始就接到很多訂房電話，4月的預約件數已是3月的1.5倍。\n不過，青森縣為了防疫，並不實

# Word Segmentation

In [8]:
pipeline.get_ws(doc)
print(doc.ws[0])
# for line in doc.ws:
#     print(line.to_text())

/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
2022-04-15 16:40:07.723647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] succ

['日本', '部分', '地區', '為', '拼', '經濟', '，', '原本', '去年', '4月', '起', '實施', '的', '「', '縣民', '優惠', '」', '措施', '，', '4月', '1日', '起', '擴大為', '「', '區域', '優惠', '」', '方案', '；', '部分', '縣府', '主管', '說', '，', '盼', '趁著', '第7', '波', '疫情', '來臨', '前', '趕緊', '實施', '，', '降低', '對', '觀光業', '的', '衝擊', '。\n', '日本', '讀賣', '新聞', '報導', '，', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾病', '）', '疫情', '對', '觀光業', '帶來', '嚴重', '打擊', '，', '日本', '有些', '縣', '的', '優惠', '措施', '，', '僅', '限', '居住', '在', '縣', '內', '或', '鄰縣', '的', '民眾', '，', '但', '1日', '起', '擴大為', '區域', '優惠', '，', '平均', '每', '人', '每', '晚', '住宿費', '最多', '可', '獲', '7000', '日圓', '（', '約', '新台幣', '1665', '元', '）', '補助', '，', '一直', '到', '28日', '為止', '。\n', '區域', '優惠', '的', '範圍', '分', '6', '大', '區', '，', '北海道', '暨', '東北', '（', '涵蓋', '北海道', '、', '青森', '、', '岩手', '、', '宮城', '、', '秋田', '、', '山形', '、', '福島縣', '）', '、', '關東', '（', '涵蓋茨城', '、', '櫪木', '、', '群馬', '、', '埼玉', '、', '千葉', '、', '東京', '、', '神奈川', '、', '山梨', '）', '、', '北陸', '信越', '暨', '中部', '（', '涵蓋', '新潟', '、', '富山', '、

In [9]:
# Named-Entity Recognition
pipeline.get_ner(doc)
print(type(doc.ner[0]))
print(doc.ner[0])

/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/home/oud/anaconda3/envs/projectV/lib/python3.8/site-packages/ckiptagger/model_ner.py:57: Us

<class 'ckipnlp.container.ner.NerSentence'>
[NerToken(word='神奈川縣', ner='GPE', idx=(727, 731)), NerToken(word='長野', ner='GPE', idx=(299, 301)), NerToken(word='中國', ner='GPE', idx=(325, 327)), NerToken(word='2020年7月22日', ner='DATE', idx=(350, 360)), NerToken(word='岐阜', ner='GPE', idx=(305, 307)), NerToken(word='去年4月', ner='DATE', idx=(476, 480)), NerToken(word='山梨縣', ner='GPE', idx=(707, 710)), NerToken(word='九州', ner='GPE', idx=(331, 333)), NerToken(word='千葉', ner='GPE', idx=(266, 268)), NerToken(word='靜岡', ner='GPE', idx=(308, 310)), NerToken(word='約7成', ner='PERCENT', idx=(717, 720)), NerToken(word='去年4月', ner='DATE', idx=(13, 17)), NerToken(word='東京', ner='GPE', idx=(269, 271)), NerToken(word='6', ner='CARDINAL', idx=(597, 598)), NerToken(word='沖繩', ner='GPE', idx=(334, 336)), NerToken(word='3000日圓', ner='MONEY', idx=(890, 896)), NerToken(word='2020年底', ner='DATE', idx=(399, 405)), NerToken(word='1.5', ner='CARDINAL', idx=(954, 957)), NerToken(word='7000日圓', ner='MONEY', idx=(843, 84

# NER

In [10]:
type(doc.ner)
doc.ner[0][0]

NerToken(word='神奈川縣', ner='GPE', idx=(727, 731))

In [11]:
ner = pd.DataFrame(doc.ner[0])
ner

,word,ner,idx
0,神奈川縣,GPE,"(727, 731)"
1,長野,GPE,"(299, 301)"
2,中國,GPE,"(325, 327)"
3,2020年7月22日,DATE,"(350, 360)"
4,岐阜,GPE,"(305, 307)"
...,...,...,...
77,早上,TIME,"(928, 930)"
78,關東,GPE,"(249, 251)"
79,關東,LOC,"(732, 734)"
80,秩父市,GPE,"(905, 908)"


# Filter

In [75]:
# Filter dataframe from NER types that we want only.
ner_filtered = ner.loc[ner['ner'].str.contains('ORG|EVENT|FAC|NORP|PERSON|PRODUCT|WORK_OF_ART|GPE|LOC',flags=re.I,regex=True)]

# Keep only duplicated words
ner_filtered = ner_filtered[ner_filtered.duplicated('word')]

In [76]:
ner_filtered

,word,ner,idx
30,日本,GPE,"(124, 126)"
32,日本,NORP,"(563, 565)"
37,北海道,GPE,"(225, 228)"
40,日本,GPE,"(0, 2)"
45,日本,NORP,"(82, 84)"
51,山梨縣,GPE,"(757, 760)"
54,東京都,GPE,"(1030, 1033)"
57,日本,GPE,"(341, 343)"
63,群馬,GPE,"(775, 777)"
68,東京都,GPE,"(723, 726)"


In [77]:
ner_filtered.drop_duplicates("word", keep="first", inplace=True)
ner_filtered.reset_index(drop=True, inplace=True)

In [78]:
ner_filtered

,word,ner,idx
0,日本,GPE,"(124, 126)"
1,北海道,GPE,"(225, 228)"
2,山梨縣,GPE,"(757, 760)"
3,東京都,GPE,"(1030, 1033)"
4,群馬,GPE,"(775, 777)"
5,埼玉,GPE,"(263, 265)"
6,關東,LOC,"(732, 734)"
